In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
import csv, os
import requests
from lxml import etree
import re

In [2]:
def get_answer(answer):
    texts = answer.find_element(By.CLASS_NAME,'RichContent-inner').text.replace('\n','')

    upvote_count = answer.find_element(By.XPATH,".//div[@class='ContentItem AnswerItem']/meta[@itemprop='upvoteCount']").get_attribute('content')
    url = answer.find_element(By.XPATH,".//div[@class='ContentItem AnswerItem']/meta[@itemprop='url']").get_attribute('content')
    time_modified = answer.find_element(By.XPATH,".//div[@class='ContentItem AnswerItem']/meta[@itemprop='dateModified']").get_attribute('content')
    time_created = answer.find_element(By.XPATH,".//div[@class='ContentItem AnswerItem']/meta[@itemprop='dateCreated']").get_attribute('content')
    comment_count = answer.find_element(By.XPATH,".//div[@class='ContentItem AnswerItem']/meta[@itemprop='commentCount']").get_attribute('content')

    author_url = answer.find_element(By.XPATH,".//div[@class='AuthorInfo']/meta[@itemprop='url']").get_attribute('content')
    res = requests.get(author_url)
    if gender := re.search(r'<meta itemProp="gender" content="(\w+)"',res.text, re.IGNORECASE):
        gender = gender.group(1)
    if followers := re.search(r'<meta itemProp="zhihu:followerCount" content="(\d*)"',res.text, re.IGNORECASE):
        followers = int(followers.group(1))
    if location := re.search(r'"ipInfo":"IP 属地(.*)","vip',res.text, re.IGNORECASE):
        location = location.group(1)
    if following := re.search(r'关注了</div><strong class="NumberBoard-itemValue" title="(\d+)">',res.text, re.IGNORECASE):
        following = int(following.group(1))
    
    info = {'texts':texts, 'url':url, 'upvote_count':upvote_count,'time_modified':time_modified,
            'comment_count':comment_count,'author_info':author_url,
            'author_gender':gender,'author_followers':followers,'author_following':following,'author_location':location}
    return info

In [3]:
js="window.scrollTo(0,document.body.scrollHeight)"
savefile = 'zhihu/zhihu.csv'

In [8]:
with open('zhihu/Zhihu_completed.txt', 'r') as f:
    urls_finished = f.readlines()  # Read all lines in the file
urls_finished = [url.strip() for url in urls_finished]

with open('zhihu/Zhihu_all_questions_urls.txt', 'r') as f:
    urls = f.readlines()  # Read all lines in the file
urls = [url.strip() for url in urls]
urls = [url for url in urls if url not in urls_finished]
len(urls)

35

In [9]:
with open(savefile, 'a+') as f:
    header = ['title','question_url','total_answers','index','texts','url','upvote_count','time_modified',
              'comment_count','author_info','author_gender','author_followers','author_following','author_location']
    writer = csv.DictWriter(f, delimiter=',', lineterminator='\n',fieldnames=header)
    if not os.path.getsize(savefile):
        writer.writeheader()

    for url in urls:
        print(url)
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)

        # close the login in window (doesn't influence scraping)
        driver.find_element(By.XPATH, "//button[@aria-label='关闭'][@type='button']").click()

        # get meta info
        title = driver.find_element(By.XPATH, "//meta[@itemprop='name']").get_attribute('content')
        num_str = driver.find_element(By.XPATH, "//h4[@class='List-headerText']").text
        num = int(num_str.split(' ')[0])

        i = 0
        while(1):
            while(1):
                try:
                    answer = driver.find_element(By.XPATH, f"//div[@class='List-item'][{i+1}]")
                except:
                    break
                info = get_answer(answer)
                info['title'] = title
                info['question_url'] = url
                info['total_answers'] = num
                info['index'] = i
                writer.writerow(info)
                #save_data = save_data.append(info,ignore_index=True)
                i += 1
                if i == 3: break

            if driver.find_elements(By.XPATH, "//div[@data-za-detail-view-path-module='BottomBar']"):
                print(i)
                break
            driver.execute_script(js)
            driver.execute_script('window.scrollBy(0,-500)')
            driver.execute_script(js)
            sleep(5)
        with open('zhihu/Zhihu_completed.txt', 'a+') as fi:
            fi.write(url+'\n')
        sleep(30)

https://www.zhihu.com/question/505604652



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST chromedriver version for 113.0.5672 google-chrome
[WDM] - Driver [/Users/liaoyufang/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache
/var/folders/8v/xpk1jpgx2kbgxh9xl6dwblt00000gp/T/ipykernel_70978/3750717278.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


2
https://www.zhihu.com/question/465941960



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST chromedriver version for 113.0.5672 google-chrome
[WDM] - Driver [/Users/liaoyufang/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache


3
https://www.zhihu.com/question/461626026



...

7
https://www.zhihu.com/question/388923063



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST chromedriver version for 113.0.5672 google-chrome
[WDM] - Driver [/Users/liaoyufang/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache


20
https://www.zhihu.com/question/489848447



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 113.0.5672
[WDM] - Get LATEST chromedriver version for 113.0.5672 google-chrome
[WDM] - Driver [/Users/liaoyufang/.wdm/drivers/chromedriver/mac64/113.0.5672.63/chromedriver] found in cache


182
